In [1]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, asin, sqrt, pi, atan, atan2, fabs
import time
from arbol_n_dimensiones import Arbol_n_dimensiones

In [2]:
def calcular_distancia(lat1,long1,lat2,long2):
    lon1, lat1 = (radians(coord) for coord in (long1,lat1))
    lon2, lat2 = (radians(coord) for coord in (long2,lat2))
    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)
    a = (
        sin(dlat * 0.5)**2 +
        cos(lat1) * cos(lat2) * sin(dlon * 0.5)**2
    )
    
    radioTierra = 6371008.8
    return 2 * radioTierra * asin(sqrt(a)) #lo dejo en metros

#distancia de Haversine
#https://gist.github.com/habibutsu/8bbcc202a915e965c6a6d4f561d0e482

In [40]:
propiedades = pd.read_csv('/home/mati/Desktop/properati.csv')
#propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/properati.csv')

In [4]:
propiedades = propiedades.loc[propiedades.Year >= 2017,:]

In [5]:
propiedades = propiedades.loc[(propiedades.lat.notnull()) & (propiedades.lon.notnull()) &\
                             (propiedades.superficie.notnull()) & (propiedades.price_aprox_usd.notnull()),:]
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294660 entries, 1988104 to 2455308
Data columns (total 23 columns):
created_on                    294660 non-null object
currency                      293968 non-null object
expenses                      41414 non-null float64
fecha_de_publicacion          294660 non-null object
floor                         36468 non-null float64
lat                           294660 non-null float64
lon                           294660 non-null float64
place_name                    294596 non-null object
place_with_parent_names       294660 non-null object
price                         294660 non-null float64
price_aprox_local_currency    294660 non-null float64
price_aprox_usd               294660 non-null float64
price_per_m2                  282871 non-null float64
price_usd_per_m2              294660 non-null float64
property_type                 294660 non-null object
rooms                         198414 non-null float64
state_name                

## Tomo como set de prueba las publicaciones de junio 2017

In [6]:
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),:].reset_index()
set_datos = propiedades.loc[(propiedades.Year != 2017) | (propiedades.Month < 6),:].reset_index()

In [7]:
set_pruebas = set_pruebas.sample(14166).reset_index()
set_pruebas = set_pruebas.loc[:,['property_type','lat','lon','superficie','price_aprox_usd']]
set_pruebas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 5 columns):
property_type      14166 non-null object
lat                14166 non-null float64
lon                14166 non-null float64
superficie         14166 non-null float64
price_aprox_usd    14166 non-null float64
dtypes: float64(4), object(1)
memory usage: 553.4+ KB


In [8]:
set_datos = set_datos.loc[:,['property_type','lat','lon','superficie','price_aprox_usd']]
set_datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232005 entries, 0 to 232004
Data columns (total 5 columns):
property_type      232005 non-null object
lat                232005 non-null float64
lon                232005 non-null float64
superficie         232005 non-null float64
price_aprox_usd    232005 non-null float64
dtypes: float64(4), object(1)
memory usage: 8.9+ MB


### Agrupo por tipo de propiedad y luego por superficie, serian los lideres de knn

In [30]:
def redondear_superficie(superficie):
    """Convierto la superficie a un multiplo de 10"""
    superficie /= 10
    superficie = round(superficie) #lo hago entero
    superficie *= 10
    return int(superficie)

In [10]:
set_datos.loc[:,'superficie_actualizada'] = set_datos.loc[:,'superficie'].apply(redondear_superficie)
set_datos.head(2)

,property_type,lat,lon,superficie,price_aprox_usd,superficie_actualizada
0,house,-34.490998,-58.76748,300.0,110000.0,300
1,apartment,-34.587785,-58.38658,162.0,645000.0,160


In [11]:
set_datos_agrupados = set_datos.groupby('property_type')

In [12]:
minimo_datos_en_hoja = 2000

In [13]:
#Agrupo por superficie cada grupo
grupos = {}
for grupo in set_datos_agrupados.__iter__():
    clave = grupo[0]
    print(clave)
    valor = grupo[1].groupby('superficie_actualizada')
    diccionario = {}
    for group in valor.__iter__():
        sub_clave = group[0]
        dataframe = group[1]
        datos = Arbol_n_dimensiones(dataframe, ['lat','lon'], minimo_datos_en_hoja)
        diccionario[sub_clave] = datos
    grupos[clave] = diccionario

PH
apartment
house
store


In [14]:
set_datos_agrupados = grupos

#### Los datos quedan dic = {prop_type: {sup : arbol n dimensiones}}

## Algoritmo de KNN

In [15]:
def calcular_distancias_para_knn(datos,prueba):
    grupo = datos[prueba.property_type]
    prueba_sup = redondear_superficie(prueba.superficie)
    analizados = []
    for sup in range(prueba_sup - 10, prueba_sup + 20, 10):  #Miro sup +- 10
        try:
            a_analizar = grupo[sup].obtener_dataframe(prueba) #Lo busco en el arbol
            for i in range(len(a_analizar)):
                distancia = calcular_distancia(a_analizar.lat[i],a_analizar.lon[i],prueba.lat,prueba.lon)
                analizados.append((distancia,a_analizar.price_aprox_usd[i]))
        except:
            #Si no hay ninguna propiedad con superficie sup
            continue
    return analizados

In [16]:
def knn(datos,pruebas,k):
    for i in range(len(pruebas)):
        distancias = calcular_distancias_para_knn(datos,pruebas.iloc[i])
        resultado = obtener_promedio_k_minimos(distancias,k)
        pruebas.resultado[i] = resultado
        pruebas.cantidad_analizados[i] = len(distancias)
    return pruebas

In [17]:
def knn_con_precision(datos,pruebas,lista_k): #Para buscar el mejor K de una lista
    resultados = {}
    for i in range(len(pruebas)):
        distancias = calcular_distancias_para_knn(datos,pruebas.iloc[i])
        resultados = obtener_promedio_k_minimos(resultados,pruebas.price_aprox_usd[i],distancias,lista_k)
    return resultados

In [18]:
def obtener_promedio_k_minimos(resultados,real,distancias,lista_k):
    distancias.sort()
    for k in lista_k:
        k2 = min(k,len(distancias))
        a_analizar = distancias[:k2]
        suma = 0
        total = 0
        for x in range(len(a_analizar)):
            suma += a_analizar[x][1]
            total += 1
        if total != 0:
            res = suma / total
        else:
            res = 0
        lista = resultados.get(k,[])
        lista.append((real,res))
        resultados[k] = lista
    return resultados

In [19]:
def calcular_precision(resultados):
    res = {}
    for k in resultados.keys():
        suma = 0
        total = 0
        lista = resultados[k]
        for tupla in lista:
            suma += calcular_resultado(tupla[0],tupla[1])
            total += 1
        res[k] = round(suma*100 / total,2)
    return res

In [20]:
def calcular_resultado(real,calculado):
    porcentaje = 0.025
    if abs(real - calculado) <= real * porcentaje:
        return 1
    return 0

## Busqueda del hiperparametro K

##### Empezamos con algunos k salteados

In [36]:
lista_k = [5,10,20,30,50,100,200,350,500,750,1000]

inicio = time.strftime("%H:%M:%S")
pruebas = knn_con_precision(set_datos_agrupados,set_pruebas,lista_k)
precision = calcular_precision(pruebas)
print("Inicio: ({}) Final: ({})\n".format(inicio,time.strftime("%H:%M:%S")))
for k in lista_k:
    print("Para k = {} la precision es {}%".format(k,precision[k]))


Inicio: (20:18:27) Final: (22:16:45)

Para k = 5 la precision es 13.38%
Para k = 10 la precision es 5.39%
Para k = 20 la precision es 3.8%
Para k = 30 la precision es 3.48%
Para k = 50 la precision es 3.19%
Para k = 100 la precision es 2.9%
Para k = 200 la precision es 2.56%
Para k = 350 la precision es 2.39%
Para k = 500 la precision es 2.33%
Para k = 750 la precision es 2.39%
Para k = 1000 la precision es 2.18%


### Se ve que la precision es mas alta para valores bajos de k
##### Ajustamos la busqueda

In [21]:
lista_k = list(range(1,50))

inicio = time.strftime("%H:%M:%S")
pruebas = knn_con_precision(set_datos_agrupados,set_pruebas,lista_k)
precision = calcular_precision(pruebas)
print("Inicio: ({}) Final: ({})\n".format(inicio,time.strftime("%H:%M:%S")))
for k in lista_k:
    print("Para k = {} la precision es {}%".format(k,precision[k]))


Inicio: (09:42:17) Final: (11:41:23)

Para k = 1 la precision es 26.54%
Para k = 2 la precision es 20.27%
Para k = 3 la precision es 18.6%
Para k = 4 la precision es 16.05%
Para k = 5 la precision es 13.38%
Para k = 6 la precision es 10.68%
Para k = 7 la precision es 8.0%
Para k = 8 la precision es 6.86%
Para k = 9 la precision es 6.08%
Para k = 10 la precision es 5.39%
Para k = 11 la precision es 5.11%
Para k = 12 la precision es 4.81%
Para k = 13 la precision es 4.47%
Para k = 14 la precision es 4.38%
Para k = 15 la precision es 4.22%
Para k = 16 la precision es 4.21%
Para k = 17 la precision es 3.97%
Para k = 18 la precision es 3.97%
Para k = 19 la precision es 3.8%
Para k = 20 la precision es 3.73%
Para k = 21 la precision es 3.67%
Para k = 22 la precision es 3.56%
Para k = 23 la precision es 3.54%
Para k = 24 la precision es 3.41%
Para k = 25 la precision es 3.4%
Para k = 26 la precision es 3.32%
Para k = 27 la precision es 3.29%
Para k = 28 la precision es 3.28%
Para k = 29 la pr

## A ver que pasa con mas datos 
#### Tomaremos datos de 2017 y mitad de 2016

In [25]:
propiedades = propiedades.loc[propiedades.Year >= 2016,:]
propiedades = propiedades.loc[(propiedades.Year == 2017) | (propiedades.Month > 6),:]
propiedades = propiedades.loc[(propiedades.lat.notnull()) & (propiedades.lon.notnull()) &\
                             (propiedades.superficie.notnull()) & (propiedades.price_aprox_usd.notnull()),:]
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 431899 entries, 1771318 to 2455308
Data columns (total 23 columns):
created_on                    431899 non-null object
currency                      428499 non-null object
expenses                      64126 non-null float64
fecha_de_publicacion          431899 non-null object
floor                         53484 non-null float64
lat                           431899 non-null float64
lon                           431899 non-null float64
place_name                    431779 non-null object
place_with_parent_names       431899 non-null object
price                         431899 non-null float64
price_aprox_local_currency    431899 non-null float64
price_aprox_usd               431899 non-null float64
price_per_m2                  411278 non-null float64
price_usd_per_m2              431899 non-null float64
property_type                 431899 non-null object
rooms                         293609 non-null float64
state_name                

In [36]:
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),:].reset_index()
set_datos = propiedades.loc[(propiedades.Year != 2017) | (propiedades.Month < 6),:].reset_index()

In [37]:
set_pruebas = set_pruebas.sample(14166).reset_index()
set_pruebas = set_pruebas.loc[:,['property_type','lat','lon','superficie','price_aprox_usd']]
set_datos = set_datos.loc[:,['property_type','lat','lon','superficie','price_aprox_usd']]
set_datos.loc[:,'superficie_actualizada'] = set_datos.loc[:,'superficie'].apply(redondear_superficie)
set_datos_agrupados = set_datos.groupby('property_type')
minimo_datos_en_hoja = 2000

In [38]:
#Agrupo por superficie cada grupo
grupos = {}
for grupo in set_datos_agrupados.__iter__():
    clave = grupo[0]
    print(clave)
    valor = grupo[1].groupby('superficie_actualizada')
    diccionario = {}
    for group in valor.__iter__():
        sub_clave = group[0]
        dataframe = group[1]
        datos = Arbol_n_dimensiones(dataframe, ['lat','lon'], minimo_datos_en_hoja)
        diccionario[sub_clave] = datos
    grupos[clave] = diccionario
    
set_datos_agrupados = grupos

PH
apartment
house
store


In [39]:
lista_k = [5,10,20,30,50,100,200,350,500,750,1000]

inicio = time.strftime("%H:%M:%S")
pruebas = knn_con_precision(set_datos_agrupados,set_pruebas,lista_k)
precision = calcular_precision(pruebas)
print("Inicio: ({}) Final: ({})\n".format(inicio,time.strftime("%H:%M:%S")))
for k in lista_k:
    print("Para k = {} la precision es {}%".format(k,precision[k]))

Inicio: (12:47:43) Final: (13:42:35)

Para k = 5 la precision es 9.45%
Para k = 10 la precision es 3.69%
Para k = 20 la precision es 2.22%
Para k = 30 la precision es 2.09%
Para k = 50 la precision es 1.46%
Para k = 100 la precision es 1.35%
Para k = 200 la precision es 1.32%
Para k = 350 la precision es 1.19%
Para k = 500 la precision es 1.17%
Para k = 750 la precision es 1.06%
Para k = 1000 la precision es 1.02%


##### Empeora la precision
### Pruebo con menos datos

In [41]:
propiedades = propiedades.loc[propiedades.Year >= 2017,:]
propiedades = propiedades.loc[(propiedades.Month > 2),:]  #marzo,abril,mayo,junio
propiedades = propiedades.loc[(propiedades.lat.notnull()) & (propiedades.lon.notnull()) &\
                             (propiedades.superficie.notnull()) & (propiedades.price_aprox_usd.notnull()),:]
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236370 entries, 2078204 to 2455308
Data columns (total 23 columns):
created_on                    236370 non-null object
currency                      236231 non-null object
expenses                      33564 non-null float64
fecha_de_publicacion          236370 non-null object
floor                         29690 non-null float64
lat                           236370 non-null float64
lon                           236370 non-null float64
place_name                    236324 non-null object
place_with_parent_names       236370 non-null object
price                         236370 non-null float64
price_aprox_local_currency    236370 non-null float64
price_aprox_usd               236370 non-null float64
price_per_m2                  227270 non-null float64
price_usd_per_m2              236370 non-null float64
property_type                 236370 non-null object
rooms                         158200 non-null float64
state_name                

In [42]:
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),:].reset_index()
set_datos = propiedades.loc[(propiedades.Year != 2017) | (propiedades.Month < 6),:].reset_index()

In [43]:
set_pruebas = set_pruebas.sample(14166).reset_index()
set_pruebas = set_pruebas.loc[:,['property_type','lat','lon','superficie','price_aprox_usd']]
set_datos = set_datos.loc[:,['property_type','lat','lon','superficie','price_aprox_usd']]
set_datos.loc[:,'superficie_actualizada'] = set_datos.loc[:,'superficie'].apply(redondear_superficie)
set_datos_agrupados = set_datos.groupby('property_type')
minimo_datos_en_hoja = 2000

In [44]:
#Agrupo por superficie cada grupo
grupos = {}
for grupo in set_datos_agrupados.__iter__():
    clave = grupo[0]
    print(clave)
    valor = grupo[1].groupby('superficie_actualizada')
    diccionario = {}
    for group in valor.__iter__():
        sub_clave = group[0]
        dataframe = group[1]
        datos = Arbol_n_dimensiones(dataframe, ['lat','lon'], minimo_datos_en_hoja)
        diccionario[sub_clave] = datos
    grupos[clave] = diccionario
    
set_datos_agrupados = grupos

PH
apartment
house
store


In [45]:
lista_k = [5,10,20,30,50,100,200,350,500,750,1000]

inicio = time.strftime("%H:%M:%S")
pruebas = knn_con_precision(set_datos_agrupados,set_pruebas,lista_k)
precision = calcular_precision(pruebas)
print("Inicio: ({}) Final: ({})\n".format(inicio,time.strftime("%H:%M:%S")))
for k in lista_k:
    print("Para k = {} la precision es {}%".format(k,precision[k]))

Inicio: (13:52:48) Final: (16:23:23)

Para k = 5 la precision es 10.44%
Para k = 10 la precision es 5.76%
Para k = 20 la precision es 4.45%
Para k = 30 la precision es 3.89%
Para k = 50 la precision es 3.67%
Para k = 100 la precision es 3.49%
Para k = 200 la precision es 3.57%
Para k = 350 la precision es 3.47%
Para k = 500 la precision es 3.37%
Para k = 750 la precision es 3.02%
Para k = 1000 la precision es 3.18%
